In [ ]:
import tensorflow as tf 
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import pathlib 
import random

In [ ]:
path = '../input/alzheimer-mri-dataset/Dataset/'
data_dir = pathlib.Path(path)
class_names = np.array([sorted(item.name for item in data_dir.glob("*"))])
print(class_names)

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.utils import image_dataset_from_directory
batch_size = 32
img_height = 128
img_width = 128

from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.utils import image_dataset_from_directory

train_data = image_dataset_from_directory(
                  data_dir,
                  validation_split=0.2,
                  subset="training",
                  seed=123,
                  image_size=(img_height, img_width),
                  batch_size=batch_size)


val_data = image_dataset_from_directory(data_dir,
                                        validation_split=0.2,
                                        subset="validation",
                                        seed=123,
                                        image_size=(img_height,img_width),
                                        batch_size=batch_size)

In [ ]:
# load model
model = VGG16()
# summarize the model
model.summary()

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import  RMSprop, Adam, Adamax

num_class = 4  # Cantidad de clases a clasificar


base_model = VGG16(include_top=False, weights='imagenet', input_shape=(128,128,3))

# COMIENZO A AGREGAR DISTINTAS CAPAS
# NOTA: ESTO TAMBIEN SE PUEDE HACER MEDIATE UNA RED SECUENCIAL
x = base_model.output
# AGREGAMOS ALGUNAS CAPAS A LA RED BASE
x = GlobalAveragePooling2D(name="GAP2D_Al")(x)
x = Dense(512,activation='relu', name="Dense1_Al")(x)
x = Dropout(0.5, name="Drop1_Al")(x)
x = Dense(512,activation='tanh', name="Dense2_Al")(x) 
x = Dropout(0.5, name="Drop2_Al")(x)
# FINALMENTE AGREGO UNA CAPA SOFTMAX CON 4 CLASES
preds = Dense(num_class, activation='softmax', name="Softmax1_Al")(x) 

# ARMO EL MODELO COMPLETO
model = Model(inputs=base_model.input, outputs=preds)


# Freeze four convolution blocks
for layer in model.layers[:19]:
    layer.trainable = False
# Make sure you have frozen the correct layers
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

model.summary()

In [ ]:
# DEFINO LA CANTIDAD DE EPOCAS DE ENTRENAMIENTO
# ATENCION! N0 HAGO EARLY STOP
epochs = 60
# DEFINO LA TASA DE APRENDIZAJE
learning_rate = 0.001
# 00 - lr = 0.0005
# 01 - lr = 0.001

# DEFINO LA TASA DE CAIDA
# decay_rate = learning_rate / epochs
#decay_rate = 0.01
# 00 - decay = learning_rate / epochs
# 01 - decay = 0.01
# 02 - decay = 0.001

# DEFINO EL OPTIMIZADOR
# opt = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=decay_rate, amsgrad=False)
opt = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=0.1, amsgrad=False)
#opt2 = Adamax( learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=0.02)
# COMPILO EL MODELO - USO CATEGORICAL CROSSENTROPY COMO LOSS FUNCTION
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['categorical_accuracy'])

history = model.fit(train_data,
                    epochs=epochs,
                    validation_data=val_data, 
                    batch_size=batch_size)

In [ ]:
acc = history.history['categorical_accuracy']    


loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training categorical_accuracy')

plt.title('Training accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')

plt.title('Training  loss')
plt.legend()

plt.show()